In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Read in energy generation data as dataframe
generation_df = pd.read_csv('Data/energy_generation_data.csv', index_col=0, parse_dates=True)

generation_df.head()

,coal,natural gas,nuclear,petroleum,other,solar,hydro,wind
date,,,,,,,,
2023-12-31,1870097,4270620,2245684,7293,221858,305983,632266,1074159
2023-12-30,1964710,4369981,2215492,7661,224151,322528,628762,1064144
2023-12-29,1948705,4482785,2228321,8311,216563,312185,718066,1109628
2023-12-28,1776641,4480968,2224448,9639,209483,309365,701614,1119395
2023-12-27,1679634,4397551,2215389,8236,218310,262265,723729,1093226


In [4]:
# Read in energy demand data as dataframe
demand_df = pd.read_csv('Data/energy_demand_data.csv', index_col=0, parse_dates=True)

demand_df.head()

,total_demand
2023-12-31,10674523
2023-12-30,10821410
2023-12-29,11084647
2023-12-28,10907167
2023-12-27,10674973


In [5]:
# Sum amounts in each column of generation_df
generation_df['total_generated'] = generation_df.sum(axis=1)

generation_df['total_generated'].head()

date
2023-12-31    10627960
2023-12-30    10797429
2023-12-29    11024564
2023-12-28    10831553
2023-12-27    10598340
Name: total_generated, dtype: int64

In [8]:
# Combine dataframes
energy_df = generation_df.merge(demand_df, how='left', left_index=True, right_index=True)

energy_df.head()

,coal,natural gas,nuclear,petroleum,other,solar,hydro,wind,total_generated,total_demand
date,,,,,,,,,,
2023-12-31,1870097,4270620,2245684,7293,221858,305983,632266,1074159,10627960,10674523
2023-12-30,1964710,4369981,2215492,7661,224151,322528,628762,1064144,10797429,10821410
2023-12-29,1948705,4482785,2228321,8311,216563,312185,718066,1109628,11024564,11084647
2023-12-28,1776641,4480968,2224448,9639,209483,309365,701614,1119395,10831553,10907167
2023-12-27,1679634,4397551,2215389,8236,218310,262265,723729,1093226,10598340,10674973


In [9]:
# Check null values
energy_df.isna().sum()

coal               0
natural gas        0
nuclear            0
petroleum          0
other              0
solar              0
hydro              0
wind               0
total_generated    0
total_demand       0
dtype: int64

In [10]:
# Get summary statistics for entire dataframe
energy_df.describe()

,coal,natural gas,nuclear,petroleum,other,solar,hydro,wind,total_generated,total_demand
count,1.826000e+03,1.826000e+03,1.826000e+03,1826.000000,1826.000000,1826.000000,1.826000e+03,1.826000e+03,1.826000e+03,1.826000e+03
mean,2.271783e+06,4.156591e+06,2.165472e+06,22896.248083,207222.064622,283472.796824,7.364040e+05,1.021341e+06,1.086518e+07,1.103157e+07
std,5.637212e+05,8.559362e+05,1.624858e+05,17994.892259,36432.085530,117613.939261,1.212391e+05,3.518294e+05,1.301253e+06,1.304928e+06
min,1.136578e+06,2.362626e+06,1.749670e+06,2959.000000,132441.000000,46755.000000,4.033230e+05,2.900780e+05,8.238613e+06,8.404965e+06
25%,1.838388e+06,3.497317e+06,2.056667e+06,12411.000000,183975.750000,194812.250000,6.574418e+05,7.586960e+05,9.893677e+06,1.002840e+07
50%,2.186894e+06,4.001372e+06,2.197422e+06,22649.500000,203175.000000,261994.000000,7.339280e+05,9.923050e+05,1.058130e+07,1.076194e+07
75%,2.701618e+06,4.698168e+06,2.272156e+06,27260.750000,225769.500000,358195.250000,8.149050e+05,1.241677e+06,1.176907e+07,1.194191e+07
max,3.962782e+06,6.658612e+06,2.509082e+06,427809.000000,672637.000000,587923.000000,1.122145e+06,2.076086e+06,1.469991e+07,1.482975e+07


In [11]:
# Check if there are entries where total_generated >= total_demand
energy_df[energy_df['total_generated'] >= energy_df['total_demand']]

,coal,natural gas,nuclear,petroleum,other,solar,hydro,wind,total_generated,total_demand
date,,,,,,,,,,
2023-12-14,1966272,4716258,2215313,10894,212605,287687,677762,1280292,11367083,11343308
2023-12-13,2142857,4826811,2214804,10446,207415,255117,693142,1112513,11463105,11420225
2023-12-12,2122030,4857500,2228112,11894,204378,316635,690244,1126899,11557692,11538087
2023-12-11,2115498,5048076,2212048,9133,201834,337969,658876,954481,11537915,11492941
2023-12-10,1758564,4241941,2213835,8274,209189,277478,538059,1135502,10382842,10340024
2023-12-09,1425607,3544286,2220954,8007,204384,302981,539085,1908785,10154089,10077066
2023-12-08,1647986,3951326,2262376,9378,205692,289081,578368,1771446,10715653,10676745
2023-12-07,1756944,4142559,2274760,9019,218738,305698,581667,1963765,11253150,11209022
2023-12-06,1841018,4490969,2270156,9330,206133,345397,585871,1474400,11223274,11180718


In [15]:
more_generated = len(energy_df[energy_df['total_generated'] >= energy_df['total_demand']])

print(f'''
    Surplus Energy Days: {more_generated}
    Deficit Energy Days: {len(energy_df) - more_generated}
''')


    Surplus Energy Days: 31
    Deficit Energy Days: 1795

